<a href="https://colab.research.google.com/github/ssjamthe/stockpredictiondags/blob/master/StockPredictV2SequenceRecords_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os
import pandas as pd
import numpy as np
import glob
import matplotlib
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import PIL

from sklearn.utils import shuffle
# import tensorflow_model_analysis as tfma


from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import IPython.display as display
import matplotlib.pyplot as plt

In [2]:
MODEL_NAME = 'SEQUENCE_CONVOLUTIONAL'
# Possible names
# SEQUENCE_CATEGORY_EMBEDDINGS
# SEQUENCE_AVERAGE_CATEGORY_EMBEDDINGS
# SEQUENCE_CONVOLUTIONAL
# CONVOLUTIONAL_ACROSS_FEATURES
# SELF_ATTENTION
# MULTI_HEAD_SELF_ATTENTION
# SEQUENCE_CONVOLUTIONAL_MULTI_HEAD
# SEQUENCE_ONLY_CONVOLUTIONAL
# CANDLE_STICKS_ONLY_FINE_TUNING

NORMALIZE_NUMERIC_FEATURES = True
TRAIN_REPEAT_TRUE_TIMES = 1
SHUFFLE_SIZE = 3000000
RANDOMISATION_SEED = 12
ANALYSE_DATA = True
LOAD_IMAGES = False
MULTI_OUTPUT = False
SEQUENCE_LENGTH = 15

In [3]:
#stock_codes_file = '/Users/swapnil/work/stockPrediction/java/com/stockprediction/stockdownload/all_stock_codes_us.txt'
stock_codes_file = '/Users/swapnil/work/stockPrediction/java/com/stockprediction/modelinputsv2/stock_codes_to_consider.txt'
curr_time_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
saved_model_dir = "/Users/swapnil/work/stockdata/models/sequence/" + MODEL_NAME + "/singlehead/saved_models/" + curr_time_str
tensorboard_dir = "/Users/swapnil/work/stockdata/models/sequence/"+ MODEL_NAME + "/singlehead/tensorboard/" + curr_time_str
train_file = '/Users/swapnil/work/stockdata/modelsinputv2/US/tfrecords/sequence/date_splitted/five_min/2023/10/8/22/0/train/*.tfrecords'
test_file = '/Users/swapnil/work/stockdata/modelsinputv2/US/tfrecords/sequence/date_splitted/five_min/2023/10/8/22/0/test/*.tfrecords'
stats_file = '/Users/swapnil/work/stockdata/modelsinputv2/US/tfrecords/sequence/date_splitted/five_min/2023/10/8/22/0/stats/features_stats.csv'
print(tensorboard_dir)
#train_cut_off_time = 1690268400000 # 25th July 2023 00:00
train_cut_off_time=None

/Users/swapnil/work/stockdata/models/sequence/SEQUENCE_CONVOLUTIONAL/singlehead/tensorboard/20231029-005935


In [4]:
def load_all_stock_codes():
  with open(stock_codes_file) as f:
    content_list = f.readlines()
  content_list = [x.strip() for x in content_list]
  return content_list

all_stock_codes = load_all_stock_codes()
print('Total number of stock codes are ' + str(len(all_stock_codes)))

Total number of stock codes are 50


In [5]:
feature_description = {
    'stock_code': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'closing_value': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'normalised_closing_value': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'date_time': tf.io.FixedLenFeature([], tf.int64, default_value=0),

    # Simple moving average.
    'sma_5': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_fraction_more_5': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_is_price_more_5': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_fraction_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_is_price_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_13': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_fraction_more_13': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'sma_is_price_more_13': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

   # Exponential Moving average.
    'ema_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_fraction_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_is_price_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_fraction_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_is_price_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_fraction_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_is_price_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_fraction_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ema_is_price_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

    # Double EMA.
    'double_ema_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_fraction_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_is_price_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_fraction_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_is_price_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_fraction_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_is_price_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_fraction_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'double_ema_is_price_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

    # Triple EMA.
    'triple_ema_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_fraction_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_is_price_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_fraction_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_is_price_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_fraction_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_is_price_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_fraction_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'triple_ema_is_price_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

    # Modified Moving Average.
    'mma_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_fraction_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_is_price_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_fraction_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_is_price_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_fraction_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_is_price_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_fraction_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'mma_is_price_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

    # Zero-Lag Exponential Moving Average.
    'zlema_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_fraction_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_is_price_more_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_fraction_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_is_price_more_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_fraction_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_is_price_more_12': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_fraction_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'zlema_is_price_more_26': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

    # Distance from SMA.
    'distance_from_sma_5': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'distance_from_sma_8': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'distance_from_sma_13': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

   #Volume simple moving average
   'volume_sma_1': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
   'volume_sma_5': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
   'volume_sma_15': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
   'volume_sma_30': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

  # Momentum Info.
    'momentum_increasing_since_intervals': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'momentum_decreasing_since_intervals': tf.io.FixedLenFeature([], tf.int64, default_value=0),

  # MACD.
    'macd_6_13_4': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_6_13_4_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_6_13_4_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_6_13_4_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_12_26_9': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_12_26_9_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_12_26_9_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_12_26_9_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_24_52_18': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_24_52_18_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_24_52_18_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'macd_24_52_18_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

  # PPO.
    'ppo_6_13_4': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_6_13_4_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_6_13_4_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_6_13_4_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_12_26_9': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_12_26_9_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_12_26_9_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_12_26_9_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_24_52_18': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_24_52_18_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_24_52_18_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_24_52_18_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_36_78_27': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_36_78_27_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_36_78_27_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'ppo_36_78_27_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

  # PVO.
    'pvo_6_13_4': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_6_13_4_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_6_13_4_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_6_13_4_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_12_26_9': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_12_26_9_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_12_26_9_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_12_26_9_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_24_52_18': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_24_52_18_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_24_52_18_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_24_52_18_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_36_78_27': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_36_78_27_crossing_from_below': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_36_78_27_crossing_from_above': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'pvo_36_78_27_fraction_increase_than_signal': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

  # RSI.
    'rsi_2': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'rsi_4': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'rsi_6': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'rsi_14': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'rsi_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'rsi_30': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

  # Stochastic RSI.
    'stochastic_rsi_2': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'stochastic_rsi_4': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'stochastic_rsi_6': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'stochastic_rsi_14': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'stochastic_rsi_20': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'stochastic_rsi_30': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

  # Bollinger band.
    'bollinger_10_1.50': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'bollinger_20_2.00': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'bollinger_50_2.50': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),

  # Candlestick Chart.
   # 'candle_stick': tf.io.FixedLenFeature([], tf.string),

  # Labels.
    'label_at_interval_1_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_1_0.50_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_2_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_2_0.50_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_3_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_3_0.50_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_4_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_4_0.50_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_5_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_5_0.50_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_6_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_6_0.50_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_1_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_1_0.40_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_2_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_2_0.40_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_3_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_3_0.40_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_4_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_4_0.40_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_5_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_5_0.40_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_6_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_6_0.40_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_1_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_1_0.30_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_2_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_2_0.30_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_3_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_3_0.30_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_4_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_4_0.30_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_5_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_5_0.30_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_6_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_6_0.30_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_1_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_1_0.20_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_2_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_2_0.20_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_3_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_3_0.20_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_4_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_4_0.20_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_5_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_5_0.20_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_6_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_6_0.20_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_1_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_1_0.10_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_2_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_2_0.10_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_3_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_3_0.10_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_4_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_4_0.10_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_5_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_5_0.10_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),
    'label_at_interval_6_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_at_interval_6_0.10_percent_increase': tf.io.FixedLenFeature([], tf.float32, default_value=0),

    # Between points labels.
    'label_between_future_points_1_2_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_3_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_4_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_5_0.50_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_2_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_3_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_4_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_5_0.40_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_2_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_3_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_4_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_5_0.30_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_2_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_3_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_4_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_5_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_2_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_3_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_4_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_between_future_points_1_5_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),

    # Increase without decrease labels.
    'label_increase_without_decrease_1_2_0.30_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_increase_without_decrease_1_3_0.30_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_increase_without_decrease_1_4_0.30_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_increase_without_decrease_1_5_0.30_0.10_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),

    'label_increase_without_decrease_1_2_0.30_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_increase_without_decrease_1_3_0.30_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_increase_without_decrease_1_4_0.30_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'label_increase_without_decrease_1_5_0.30_0.20_is_true': tf.io.FixedLenFeature([], tf.int64, default_value=0),
}

#labels_to_consider = ['label_at_interval_4_0.30_is_true', 'label_at_interval_4_0.30_percent_increase']

labels_to_consider = ['label_increase_without_decrease_1_3_0.30_0.10_is_true']

def parse_input_data(example_proto):
  example = tf.io.parse_single_example(example_proto, feature_description)
  if LOAD_IMAGES:
    candle_stick = tf.io.decode_png(example['candle_stick'], channels = 3)
    example['candle_stick'] = candle_stick

  features = {}
  labels = {}

  for name in feature_description.keys():
    if (name == 'candle_stick' and not LOAD_IMAGES):
      continue
    if (name.startswith('label_')):
      if (name in labels_to_consider):
        labels[name] = example[name]
    else:
        features[name] = example[name]


  if (example[labels_to_consider[0]] == 1):
    sample_weight = 1
  else:
    sample_weight = 1

  if MULTI_OUTPUT:
    return (features, labels)
  else:
    return (features, labels[labels_to_consider[0]])

In [6]:
def filter_dataset(x, y):
  return x['date_time'] < train_cut_off_time

train_data = tf.data.TFRecordDataset(tf.data.TFRecordDataset.list_files(train_file))
validation_data = tf.data.TFRecordDataset(tf.data.TFRecordDataset.list_files(test_file))
train_ds = train_data.map(parse_input_data)
if train_cut_off_time != None:
  train_ds = train_ds.filter(filter_dataset)
test_ds = validation_data.map(parse_input_data)

Metal device set to: Apple M1 Max


2023-10-29 00:59:35.867208: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-29 00:59:35.867317: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
for parsed_record in train_ds.take(1):
  print(repr(parsed_record))

({'stock_code': <tf.Tensor: shape=(), dtype=string, numpy=b'VXUS'>, 'closing_value': <tf.Tensor: shape=(), dtype=float32, numpy=56.98>, 'normalised_closing_value': <tf.Tensor: shape=(), dtype=float32, numpy=0.9811321>, 'date_time': <tf.Tensor: shape=(), dtype=int64, numpy=1675335900000>, 'sma_5': <tf.Tensor: shape=(15,), dtype=float32, numpy=
array([0.72965544, 0.7089658 , 0.68689686, 0.8886794 , 0.85849077,
       0.8264153 , 0.8320757 , 0.811321  , 0.8301889 , 0.8301889 ,
       0.84150964, 0.8452832 , 0.8956605 , 0.9271699 , 0.95547175],
      dtype=float32)>, 'sma_fraction_more_5': <tf.Tensor: shape=(15,), dtype=float32, numpy=
array([-0.00188679,  0.12967013,  0.10666651,  0.19240473,  0.02247187,
       -0.01573031, -0.06170204,  0.08860744, -0.01123594, -0.01123594,
       -0.06105254, -0.06666657, -0.10179741, -0.07019858, -0.02615381],
      dtype=float32)>, 'sma_is_price_more_5': <tf.Tensor: shape=(15,), dtype=float32, numpy=
array([1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1.,

2023-10-29 00:59:36.197261: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
countries={"Ghana": "Accra", "China": "Beijing"}
# using the del keyword
print(countries)
del countries["China"]
print(countries)

{'Ghana': 'Accra', 'China': 'Beijing'}
{'Ghana': 'Accra'}


In [9]:
for parsed_record in train_ds.take(3):
  if LOAD_IMAGES:
    plt.imshow(parsed_record[0]['candle_stick'])

In [10]:
def repeat_true(ds, repeat_true_times, label):
  if MULTI_OUTPUT:
    ds_true = ds.filter(lambda x,y: y[label] == 1)
    ds_false = ds.filter(lambda x,y: y[label] == 0)
  else:
    ds_true = ds.filter(lambda x,y: y == 1)
    ds_false = ds.filter(lambda x,y: y == 0)
  ds_true = ds_true.repeat(repeat_true_times)
  ds_extended = ds_false.concatenate(ds_true)
  ds_extended = ds_extended.shuffle(SHUFFLE_SIZE, seed=RANDOMISATION_SEED)

  return ds_extended

In [11]:
train_ds = repeat_true(train_ds, TRAIN_REPEAT_TRUE_TIMES, labels_to_consider[0])
train_ds = train_ds.batch(2048)
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = test_ds.batch(2048)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [12]:
def get_category_encoding_layer_using_vocab(vocab):
  index = preprocessing.StringLookup(vocabulary = vocab)
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())
  return lambda feature: encoder(index(feature))


def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    #index = preprocessing.StringLookup(max_tokens=max_tokens)
    index = preprocessing.StringLookup()
  else:
    #index = preprocessing.IntegerLookup(max_tokens=max_tokens)
    index = preprocessing.IntegerLookup()

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])
  #feature_ds = feature_ds.shuffle(buffer_size = train_shuffle_buffer_size)

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply one-hot encoding to our indices. The lambda function captures the
  # layer so we can use them, or include them in the functional model later.
  return lambda feature: encoder(index(feature))

In [13]:
feature_stats_df = pd.read_csv(stats_file)
features_mean = {}
features_variance = {}

for _, row in feature_stats_df.iterrows():
  features_mean[row['feature_name']] = row['mean']
  features_variance[row['feature_name']] = row['variance']


In [14]:
def get_normalization_layer(name):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None, mean=features_mean[name], variance=features_variance[name])

  # Prepare a Dataset that only yields our feature.
  #feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  #normalizer.adapt(feature_ds)

  return normalizer

In [15]:
sequence_features = [
                    'sma_5',
                    'sma_fraction_more_5',
                    'sma_is_price_more_5',
                    'sma_8',
                    'sma_fraction_more_8',
                    'sma_is_price_more_8',
                    'sma_13',
                    'sma_fraction_more_13',
                    'sma_is_price_more_13',

                    'ema_8',
                    'ema_fraction_more_8',
                    'ema_is_price_more_8',
                    'ema_20',
                    'ema_fraction_more_20',
                    'ema_is_price_more_20',
                    'ema_12',
                    'ema_fraction_more_12',
                    'ema_is_price_more_12',
                    'ema_26',
                    'ema_fraction_more_26',
                    'ema_is_price_more_26',

                    'double_ema_8',
                    'double_ema_fraction_more_8',
                    'double_ema_is_price_more_8',
                    'double_ema_20',
                    'double_ema_fraction_more_20',
                    'double_ema_is_price_more_20',
                    'double_ema_12',
                    'double_ema_fraction_more_12',
                    'double_ema_is_price_more_12',
                    'double_ema_26',
                    'double_ema_fraction_more_26',
                    'double_ema_is_price_more_26',

                    'triple_ema_8',
                    'triple_ema_fraction_more_8',
                    'triple_ema_is_price_more_8',
                    'triple_ema_20',
                    'triple_ema_fraction_more_20',
                    'triple_ema_is_price_more_20',
                    'triple_ema_12',
                    'triple_ema_fraction_more_12',
                    'triple_ema_is_price_more_12',
                    'triple_ema_26',
                    'triple_ema_fraction_more_26',
                    'triple_ema_is_price_more_26',

                    'mma_8',
                    'mma_fraction_more_8',
                    'mma_is_price_more_8',
                    'mma_20',
                    'mma_fraction_more_20',
                    'mma_is_price_more_20',
                    'mma_12',
                    'mma_fraction_more_12',
                    'mma_is_price_more_12',
                    'mma_26',
                    'mma_fraction_more_26',
                    'mma_is_price_more_26',

                    'zlema_8',
                    'zlema_fraction_more_8',
                    'zlema_is_price_more_8',
                    'zlema_20',
                    'zlema_fraction_more_20',
                    'zlema_is_price_more_20',
                    'zlema_12',
                    'zlema_fraction_more_12',
                    'zlema_is_price_more_12',
                    'zlema_26',
                    'zlema_fraction_more_26',
                    'zlema_is_price_more_26',

                    'distance_from_sma_5',
                    'distance_from_sma_8',
                    'distance_from_sma_13',

                    'volume_sma_1',
                    'volume_sma_5',
                    'volume_sma_15',
                    'volume_sma_30',

                    'macd_6_13_4',
                    'macd_6_13_4_crossing_from_below',
                    'macd_6_13_4_crossing_from_above',
                    'macd_6_13_4_fraction_increase_than_signal',
                    'macd_12_26_9',
                    'macd_12_26_9_crossing_from_below',
                    'macd_12_26_9_crossing_from_above',
                    'macd_12_26_9_fraction_increase_than_signal',
                    'macd_24_52_18',
                    'macd_24_52_18_crossing_from_below',
                    'macd_24_52_18_crossing_from_above',
                    'macd_24_52_18_fraction_increase_than_signal',

                    'ppo_6_13_4',
                    'ppo_6_13_4_crossing_from_below',
                    'ppo_6_13_4_crossing_from_above',
                    'ppo_6_13_4_fraction_increase_than_signal',
                    'ppo_12_26_9',
                    'ppo_12_26_9_crossing_from_below',
                    'ppo_12_26_9_crossing_from_above',
                    'ppo_12_26_9_fraction_increase_than_signal',
                    'ppo_24_52_18',
                    'ppo_24_52_18_crossing_from_below',
                    'ppo_24_52_18_crossing_from_above',
                    'ppo_24_52_18_fraction_increase_than_signal',

                    'pvo_6_13_4',
                    'pvo_6_13_4_crossing_from_below',
                    'pvo_6_13_4_crossing_from_above',
                    'pvo_6_13_4_fraction_increase_than_signal',
                    'pvo_12_26_9',
                    'pvo_12_26_9_crossing_from_below',
                    'pvo_12_26_9_crossing_from_above',
                    'pvo_12_26_9_fraction_increase_than_signal',
                    'pvo_24_52_18',
                    'pvo_24_52_18_crossing_from_below',
                    'pvo_24_52_18_crossing_from_above',
                    'pvo_24_52_18_fraction_increase_than_signal',

                    'rsi_2',
                    'rsi_4',
                    'rsi_6',
                    'rsi_14',
                    'rsi_20',
                    'rsi_30',

                    'stochastic_rsi_2',
                    'stochastic_rsi_4',
                    'stochastic_rsi_6',
                    'stochastic_rsi_14',
                    'stochastic_rsi_20',
                    'stochastic_rsi_30',

                    'bollinger_10_1.50',
                    'bollinger_20_2.00',
                    'bollinger_50_2.50'
]

numeric_features = [#'closing_value',
                    'normalised_closing_value',
                    'momentum_increasing_since_intervals',
                    'momentum_decreasing_since_intervals',
            ]

categorical_features = ['stock_code']

numeric_features_to_normalize = [
                    'sma_5',
                    'sma_fraction_more_5',
                    'sma_is_price_more_5',
                    'sma_8',
                    'sma_fraction_more_8',
                    'sma_is_price_more_8',
                    'sma_13',
                    'sma_fraction_more_13',
                    'sma_is_price_more_13',

                    'ema_8',
                    'ema_fraction_more_8',
                    'ema_is_price_more_8',
                    'ema_20',
                    'ema_fraction_more_20',
                    'ema_is_price_more_20',
                    'ema_12',
                    'ema_fraction_more_12',
                    'ema_is_price_more_12',
                    'ema_26',
                    'ema_fraction_more_26',
                    'ema_is_price_more_26',

                    'double_ema_8',
                    'double_ema_fraction_more_8',
                    'double_ema_is_price_more_8',
                    'double_ema_20',
                    'double_ema_fraction_more_20',
                    'double_ema_is_price_more_20',
                    'double_ema_12',
                    'double_ema_fraction_more_12',
                    'double_ema_is_price_more_12',
                    'double_ema_26',
                    'double_ema_fraction_more_26',
                    'double_ema_is_price_more_26',

                    'triple_ema_8',
                    'triple_ema_fraction_more_8',
                    'triple_ema_is_price_more_8',
                    'triple_ema_20',
                    'triple_ema_fraction_more_20',
                    'triple_ema_is_price_more_20',
                    'triple_ema_12',
                    'triple_ema_fraction_more_12',
                    'triple_ema_is_price_more_12',
                    'triple_ema_26',
                    'triple_ema_fraction_more_26',
                    'triple_ema_is_price_more_26',

                    'mma_8',
                    'mma_fraction_more_8',
                    'mma_is_price_more_8',
                    'mma_20',
                    'mma_fraction_more_20',
                    'mma_is_price_more_20',
                    'mma_12',
                    'mma_fraction_more_12',
                    'mma_is_price_more_12',
                    'mma_26',
                    'mma_fraction_more_26',
                    'mma_is_price_more_26',

                    'zlema_8',
                    'zlema_fraction_more_8',
                    'zlema_is_price_more_8',
                    'zlema_20',
                    'zlema_fraction_more_20',
                    'zlema_is_price_more_20',
                    'zlema_12',
                    'zlema_fraction_more_12',
                    'zlema_is_price_more_12',
                    'zlema_26',
                    'zlema_fraction_more_26',
                    'zlema_is_price_more_26',

                    'distance_from_sma_5',
                    'distance_from_sma_8',
                    'distance_from_sma_13',

                    'volume_sma_1',
                    'volume_sma_5',
                    'volume_sma_15',
                    'volume_sma_30',

                    'macd_6_13_4',
                    'macd_6_13_4_crossing_from_below',
                    'macd_6_13_4_crossing_from_above',
                    'macd_6_13_4_fraction_increase_than_signal',
                    'macd_12_26_9',
                    'macd_12_26_9_crossing_from_below',
                    'macd_12_26_9_crossing_from_above',
                    'macd_12_26_9_fraction_increase_than_signal',
                    'macd_24_52_18',
                    'macd_24_52_18_crossing_from_below',
                    'macd_24_52_18_crossing_from_above',
                    'macd_24_52_18_fraction_increase_than_signal',

                    'ppo_6_13_4',
                    'ppo_6_13_4_crossing_from_below',
                    'ppo_6_13_4_crossing_from_above',
                    'ppo_6_13_4_fraction_increase_than_signal',
                    'ppo_12_26_9',
                    'ppo_12_26_9_crossing_from_below',
                    'ppo_12_26_9_crossing_from_above',
                    'ppo_12_26_9_fraction_increase_than_signal',
                    'ppo_24_52_18',
                    'ppo_24_52_18_crossing_from_below',
                    'ppo_24_52_18_crossing_from_above',
                    'ppo_24_52_18_fraction_increase_than_signal',

                    'pvo_6_13_4',
                    'pvo_6_13_4_crossing_from_below',
                    'pvo_6_13_4_crossing_from_above',
                    'pvo_6_13_4_fraction_increase_than_signal',
                    'pvo_12_26_9',
                    'pvo_12_26_9_crossing_from_below',
                    'pvo_12_26_9_crossing_from_above',
                    'pvo_12_26_9_fraction_increase_than_signal',
                    'pvo_24_52_18',
                    'pvo_24_52_18_crossing_from_below',
                    'pvo_24_52_18_crossing_from_above',
                    'pvo_24_52_18_fraction_increase_than_signal',

                    'rsi_2',
                    'rsi_4',
                    'rsi_6',
                    'rsi_14',
                    'rsi_20',
                    'rsi_30',

                    'stochastic_rsi_2',
                    'stochastic_rsi_4',
                    'stochastic_rsi_6',
                    'stochastic_rsi_14',
                    'stochastic_rsi_20',
                    'stochastic_rsi_30',

                    'bollinger_10_1.50',
                    'bollinger_20_2.00',
                    'bollinger_50_2.50']

In [16]:
all_sequence_inputs = []
all_numerical_inputs = []
all_categorical_inputs = []
all_image_inputs = []

encoded_numerical_inputs = []
encoded_sequence_inputs = []
encoded_categorical_inputs = []

for header in numeric_features:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  all_numerical_inputs.append(numeric_col)
  if (NORMALIZE_NUMERIC_FEATURES and header in numeric_features_to_normalize):
    normalization_layer = get_normalization_layer(header)
    encoded_numeric_col = normalization_layer(numeric_col)
    encoded_numerical_inputs.append(encoded_numeric_col)
  else:
    encoded_numerical_inputs.append(numeric_col)

for header in sequence_features:
  sequence_col = tf.keras.Input(shape=(SEQUENCE_LENGTH,1), name=header)
  all_sequence_inputs.append(sequence_col)
  if (NORMALIZE_NUMERIC_FEATURES and header in numeric_features_to_normalize):
    print("Normalizing featue " + header)
    normalization_layer = get_normalization_layer(header)
    encoded_sequence_col = normalization_layer(sequence_col)
    encoded_sequence_inputs.append(encoded_sequence_col)
  else:
    encoded_sequence_inputs.append(sequence_col)

Normalizing featue sma_5
Normalizing featue sma_fraction_more_5
Normalizing featue sma_is_price_more_5
Normalizing featue sma_8
Normalizing featue sma_fraction_more_8
Normalizing featue sma_is_price_more_8
Normalizing featue sma_13
Normalizing featue sma_fraction_more_13
Normalizing featue sma_is_price_more_13
Normalizing featue ema_8
Normalizing featue ema_fraction_more_8
Normalizing featue ema_is_price_more_8
Normalizing featue ema_20
Normalizing featue ema_fraction_more_20
Normalizing featue ema_is_price_more_20
Normalizing featue ema_12
Normalizing featue ema_fraction_more_12
Normalizing featue ema_is_price_more_12
Normalizing featue ema_26
Normalizing featue ema_fraction_more_26
Normalizing featue ema_is_price_more_26
Normalizing featue double_ema_8
Normalizing featue double_ema_fraction_more_8
Normalizing featue double_ema_is_price_more_8
Normalizing featue double_ema_20
Normalizing featue double_ema_fraction_more_20
Normalizing featue double_ema_is_price_more_20
Normalizing feat

In [17]:
for header in categorical_features:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  #encoding_layer =  get_category_encoding_layer(name=header, dataset = train_ds,dtype='string')
  encoding_layer = get_category_encoding_layer_using_vocab(all_stock_codes)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_categorical_inputs.append(categorical_col)
  encoded_categorical_inputs.append(encoded_categorical_col)

In [18]:
if LOAD_IMAGES:
  candle_stick_image_input = tf.keras.Input(shape=(256,256,3),name='candle_stick')
  all_image_inputs.append(candle_stick_image_input)

In [19]:
all_inputs = all_numerical_inputs + all_sequence_inputs + all_categorical_inputs + all_image_inputs
encoded_features = encoded_numerical_inputs + encoded_categorical_inputs
encoded_sequence_features = encoded_sequence_inputs

In [20]:
def create_single_head_sequence_model():
  features_categorical = tf.keras.layers.concatenate(encoded_categorical_inputs)
  categorical_embedding = tf.keras.layers.Dense(32, activation="relu")(features_categorical)
  categorical_embedding = tf.keras.layers.Dropout(0.2)(categorical_embedding)
  categorical_embedding = tf.keras.layers.Dense(32, activation="relu")(categorical_embedding)
  non_seq_encodings = tf.keras.layers.concatenate([categorical_embedding, tf.keras.layers.concatenate(encoded_numerical_inputs)])

  lstm = tf.keras.layers.LSTM(1024)
  seq_encodings = lstm(tf.keras.layers.concatenate(encoded_sequence_features))

  x = tf.keras.layers.concatenate([non_seq_encodings, seq_encodings])
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)

  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)
  model = tf.keras.Model(all_inputs, [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-7),
                loss={"label_at_interval_4_0.30_is_true":tf.keras.losses.BinaryCrossentropy()                },
                metrics={
                  "label_at_interval_4_0.30_is_true":metrics_increase_prob
                })
  model.summary()
  return model



In [21]:
def create_sequence_average_model():
  features_categorical = tf.keras.layers.concatenate(encoded_categorical_inputs)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(features_categorical)
  categorical_embedding = tf.keras.layers.Dropout(0.2)(categorical_embedding)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(categorical_embedding)
  non_seq_encodings = tf.keras.layers.concatenate([categorical_embedding, tf.keras.layers.concatenate(encoded_numerical_inputs)])


  avg_seq = tf.keras.layers.GlobalAveragePooling1D()(tf.keras.layers.concatenate(encoded_sequence_features))

  x = tf.keras.layers.concatenate([non_seq_encodings, avg_seq])
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)

  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)
  model = tf.keras.Model(all_inputs, [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()                },
                metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model


In [22]:
def create_sequence_only_convolutional_model():
  seq_encodings = []
  for sequnce_for_feature in encoded_sequence_features:
    seq_encodings.append(tf.keras.layers.Conv1D(128, SEQUENCE_LENGTH, padding='valid', activation='relu')(sequnce_for_feature))

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(32, activation='relu')(x)
  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)

  model = tf.keras.Model(all_sequence_inputs , [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()},
                metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model

In [23]:
# Need to resolve warning on gradients for one layer not geetin updated.
def create_sequence_convolutional_multi_head_model():
  features_categorical = tf.keras.layers.concatenate(encoded_categorical_inputs)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(features_categorical)
  categorical_embedding = tf.keras.layers.Dropout(0.2)(categorical_embedding)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(categorical_embedding)
  non_seq_encodings = tf.keras.layers.concatenate([categorical_embedding, tf.keras.layers.concatenate(encoded_numerical_inputs)])

  seq_encodings = []
  for sequnce_for_feature in encoded_sequence_features:
    seq_encodings.append(tf.keras.layers.Conv1D(128, SEQUENCE_LENGTH, padding='valid', activation='relu')(sequnce_for_feature))

  seq_encodings = tf.keras.layers.Flatten()(tf.keras.layers.concatenate(seq_encodings))

  x = tf.keras.layers.concatenate([non_seq_encodings, seq_encodings])
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  x_classification = tf.keras.layers.Dense(64, activation='relu')(x)
  output_classification = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x_classification)

  x_regression = tf.keras.layers.Dense(64, activation='relu')(x)
  output_regression = tf.keras.layers.Dense(1, name = "label_at_interval_4_0.30_percent_increase")(x_regression)

  model = tf.keras.Model(all_inputs , [output_classification, output_regression])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  metrics_increase_percent = [
    tf.keras.metrics.MeanSquaredError(name="mean_squared_error"),
    tf.keras.metrics.RootMeanSquaredError(name="root_mean_squared_error")
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy(),
                  "label_at_interval_4_0.30_percent_increase":tf.keras.metrics.MeanSquaredError()
                },
                metrics={
                  labels_to_consider[0]:metrics_increase_prob,
                  "label_at_interval_4_0.30_percent_increase":metrics_increase_percent
                })
  model.summary()
  return model

In [24]:
x = np.arange(5).reshape(5, 1)
y = np.arange(5).reshape(5, 1)

print(x)
tf.keras.layers.concatenate([x, y])

[[0]
 [1]
 [2]
 [3]
 [4]]


<tf.Tensor: shape=(5, 2), dtype=int64, numpy=
array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4]])>

In [25]:
def create_sequence_convolutional_model():
  #features_categorical = tf.keras.layers.concatenate(encoded_categorical_inputs)
  features_categorical = encoded_categorical_inputs[0]
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(features_categorical)
  categorical_embedding = tf.keras.layers.Dropout(0.2)(categorical_embedding)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(categorical_embedding)
  non_seq_encodings = tf.keras.layers.concatenate([categorical_embedding, tf.keras.layers.concatenate(encoded_numerical_inputs)])

  seq_encodings = []
  for sequnce_for_feature in encoded_sequence_features:
    seq_encodings.append(tf.keras.layers.Conv1D(128, SEQUENCE_LENGTH, padding='valid', activation='relu')(sequnce_for_feature))

  seq_encodings = tf.keras.layers.Flatten()(tf.keras.layers.concatenate(seq_encodings))

  x = tf.keras.layers.concatenate([non_seq_encodings, seq_encodings])
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)

  model = tf.keras.Model(all_inputs , [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(threshold = 0.9, name='accuracy'),
        keras.metrics.Precision(thresholds = 0.9,name='precision'),
        keras.metrics.Recall(thresholds = 0.9,name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()},
                metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model

In [26]:
def create_sequence_convolutional_across_features_model():
  features_categorical = tf.keras.layers.concatenate(encoded_categorical_inputs)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(features_categorical)
  categorical_embedding = tf.keras.layers.Dropout(0.2)(categorical_embedding)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(categorical_embedding)
  non_seq_encodings = tf.keras.layers.concatenate([categorical_embedding, tf.keras.layers.concatenate(encoded_numerical_inputs)])


  sequence_features_concatenated = tf.transpose(tf.keras.layers.concatenate(encoded_sequence_features), perm = [0, 2, 1])
  seq_encodings = tf.keras.layers.Conv1D(128, 3, padding='valid', activation='relu')(sequence_features_concatenated)
  seq_encodings = tf.keras.layers.Flatten()(seq_encodings)

  x = tf.keras.layers.concatenate([non_seq_encodings, seq_encodings])
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)

  model = tf.keras.Model(all_inputs , [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(threshold = 0.85, name='accuracy'),
        keras.metrics.Precision(thresholds = 0.85,name='precision'),
        keras.metrics.Recall(thresholds = 0.85,name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()},
                metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model

In [27]:
def create_self_attention_model():
  features_categorical = tf.keras.layers.concatenate(encoded_categorical_inputs)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(features_categorical)
  categorical_embedding = tf.keras.layers.Dropout(0.2)(categorical_embedding)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(categorical_embedding)
  non_seq_encodings = tf.keras.layers.concatenate([categorical_embedding, tf.keras.layers.concatenate(encoded_numerical_inputs)])

  sequence_features_concatenated = tf.transpose(tf.keras.layers.concatenate(encoded_sequence_features), perm = [0, 2, 1])
  query_value_attention_seq = tf.keras.layers.Attention()(
    [sequence_features_concatenated, sequence_features_concatenated])

  # Reduce over the sequence axis to produce encodings of shape
  # [batch_size, filters].
  query_encoding = tf.keras.layers.GlobalAveragePooling1D()(
    sequence_features_concatenated)
  query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(
    query_value_attention_seq)

  # Concatenate query and document encodings to produce a DNN input layer.
  seq_encodings = tf.keras.layers.Concatenate()(
    [query_encoding, query_value_attention])


  x = tf.keras.layers.concatenate([non_seq_encodings, seq_encodings])
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)

  model = tf.keras.Model(all_inputs , [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(threshold = 0.85, name='accuracy'),
        keras.metrics.Precision(thresholds = 0.85,name='precision'),
        keras.metrics.Recall(thresholds = 0.85,name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()},
                metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model


In [28]:
def create_multi_head_self_attention_model():
  features_categorical = tf.keras.layers.concatenate(encoded_categorical_inputs)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(features_categorical)
  categorical_embedding = tf.keras.layers.Dropout(0.2)(categorical_embedding)
  categorical_embedding = tf.keras.layers.Dense(8, activation="relu")(categorical_embedding)
  non_seq_encodings = tf.keras.layers.concatenate([categorical_embedding, tf.keras.layers.concatenate(encoded_numerical_inputs)])

  sequence_features_concatenated = tf.keras.layers.concatenate(encoded_sequence_features)
  query_value_attention_seq = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=len(encoded_sequence_features))(
    tf.transpose(sequence_features_concatenated, perm = [0, 2, 1]), tf.transpose(sequence_features_concatenated, perm = [0, 2, 1]))

  seq_encodings = tf.keras.layers.Conv1D(128, SEQUENCE_LENGTH, padding='valid', activation='relu')(query_value_attention_seq)
  seq_encodings = tf.keras.layers.Flatten()(seq_encodings)

  x = tf.keras.layers.concatenate([non_seq_encodings, seq_encodings])
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)

  model = tf.keras.Model(all_inputs , [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(threshold = 0.85, name='accuracy'),
        keras.metrics.Precision(thresholds = 0.85,name='precision'),
        keras.metrics.Recall(thresholds = 0.85,name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()},
                metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model


In [29]:
def create_candle_stick_only_model():
  image_input = tf.keras.Input(shape=(256,256,3),name='candle_stick')
  rescaling = tf.keras.layers.Rescaling(1./255)(image_input)
  x = tf.keras.layers.Conv2D(4, 3, padding='same', activation='relu')(rescaling)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Conv2D(8, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(32, activation='relu')(x)
  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)

  model = tf.keras.Model(image_input , [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()},
                metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model

In [30]:
def create_candle_sticks_only_fine_tuning_model():
  base_model = tf.keras.applications.MobileNetV2(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet')
  base_model.trainable = True
  # Let's take a look to see how many layers are in the base model
  print("Number of layers in the base model: ", len(base_model.layers))

  # Fine-tune from this layer onwards
  fine_tune_at = 140

  # Freeze all the layers before the `fine_tune_at` layer
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  image_input = tf.keras.Input(shape=(256,256,3),name='candle_stick')
  x = tf.keras.applications.mobilenet_v2.preprocess_input(image_input)
  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  # Extra layers.
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)

  output = tf.keras.layers.Dense(1, activation="sigmoid", name = labels_to_consider[0])(x)
  model = tf.keras.Model(image_input , [output])

  metrics_increase_prob = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
  ]

  model.compile(loss={labels_to_consider[0]:tf.keras.losses.BinaryCrossentropy()},
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-7),
              metrics={
                  labels_to_consider[0]:metrics_increase_prob
                })
  model.summary()
  return model

In [31]:
print('Model name is ' + MODEL_NAME)

if (MODEL_NAME == 'SEQUENCE_CATEGORY_EMBEDDINGS'):
  model = create_single_head_sequence_model()
elif (MODEL_NAME == 'SEQUENCE_AVERAGE_CATEGORY_EMBEDDINGS'):
  model = create_sequence_average_model()
elif (MODEL_NAME == 'SEQUENCE_ONLY_CONVOLUTIONAL'):
  model = create_sequence_only_convolutional_model()
elif (MODEL_NAME == 'SEQUENCE_CONVOLUTIONAL'):
  model = create_sequence_convolutional_model()
elif (MODEL_NAME == 'CONVOLUTIONAL_ACROSS_FEATURES'):
  model = create_sequence_convolutional_across_features_model()
elif (MODEL_NAME == 'SELF_ATTENTION'):
  model = create_self_attention_model()
elif (MODEL_NAME == 'MULTI_HEAD_SELF_ATTENTION'):
  model = create_multi_head_self_attention_model()
elif (MODEL_NAME == 'SEQUENCE_CONVOLUTIONAL_MULTI_HEAD'):
  model = create_sequence_convolutional_multi_head_model()
elif (MODEL_NAME == 'CANDLE_STICKS_ONLY_FINE_TUNING'):
  model = create_candle_sticks_only_fine_tuning_model()
elif (MODEL_NAME == 'SINGLE_HEAD_CANDLE_STICKS_ONLY'):
  model = create_candle_stick_only_model()


#tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

Model name is SEQUENCE_CONVOLUTIONAL
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 stock_code (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 string_lookup (StringLookup)   (None, 1)            0           ['stock_code[0][0]']             
                                                                                                  
 category_encoding (CategoryEnc  (None, 51)          0           ['string_lookup[0][0]']          
 oding)                                                                                           
                                                                                                  
 sma_5 (InputLayer)             [(None, 15, 1)]      0   

In [32]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
model.fit(train_ds ,epochs=10, validation_data=test_ds,  verbose=1, callbacks=[tensorboard_callback], class_weight={0:1, 1:3})

Epoch 1/10


/Users/swapnil/miniforge3/lib/python3.9/site-packages/keras/engine/functional.py:637: UserWarning: Input dict contained keys ['closing_value', 'date_time', 'ppo_36_78_27', 'ppo_36_78_27_crossing_from_below', 'ppo_36_78_27_crossing_from_above', 'ppo_36_78_27_fraction_increase_than_signal', 'pvo_36_78_27', 'pvo_36_78_27_crossing_from_below', 'pvo_36_78_27_crossing_from_above', 'pvo_36_78_27_fraction_increase_than_signal'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
2023-10-29 00:59:41.442621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-29 00:59:52.241759: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 203308 of 3000000
2023-10-29 01:00:02.241905: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 4195

    488/Unknown - 242s 275ms/step - loss: 4.2236 - tp: 27587.0000 - fp: 55680.0000 - tn: 828552.0000 - fn: 86063.0000 - accuracy: 0.8862 - precision: 0.6686 - recall: 0.0021 - auc: 0.7290 - prc: 0.2675

2023-10-29 01:03:41.725919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


488/488 [==============================] - 261s 314ms/step - loss: 4.2236 - tp: 27587.0000 - fp: 55680.0000 - tn: 828552.0000 - fn: 86063.0000 - accuracy: 0.8862 - precision: 0.6686 - recall: 0.0021 - auc: 0.7290 - prc: 0.2675 - val_loss: 1.4356 - val_tp: 5175.0000 - val_fp: 11430.0000 - val_tn: 139237.0000 - val_fn: 10736.0000 - val_accuracy: 0.9046 - val_precision: 0.6000 - val_recall: 0.0051 - val_auc: 0.7764 - val_prc: 0.2762
Epoch 2/10


...

488/488 [==============================] - 232s 281ms/step - loss: 0.5741 - tp: 45527.0000 - fp: 80134.0000 - tn: 804098.0000 - fn: 68123.0000 - accuracy: 0.8875 - precision: 0.8061 - recall: 0.0158 - auc: 0.7997 - prc: 0.3542 - val_loss: 0.7815 - val_tp: 5997.0000 - val_fp: 13037.0000 - val_tn: 137630.0000 - val_fn: 9914.0000 - val_accuracy: 0.9056 - val_precision: 0.8328 - val_recall: 0.0150 - val_auc: 0.7930 - val_prc: 0.3106
Epoch 9/10


2023-10-29 01:31:00.834930: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 234596 of 3000000
2023-10-29 01:31:10.835038: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 461300 of 3000000
2023-10-29 01:31:20.834938: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 670710 of 3000000
2023-10-29 01:31:30.837024: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 885379 of 3000000
2023-10-29 01:31:40.836120: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 898911 of 3000000
2023-10-29 01:31:50.846751: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 919051 of 3000000
2023-10-29 01:32:00.835014: I tensorflow/core/kernels/data/shuffle_dat

488/488 [==============================] - 229s 275ms/step - loss: 0.5692 - tp: 46498.0000 - fp: 80776.0000 - tn: 803456.0000 - fn: 67152.0000 - accuracy: 0.8876 - precision: 0.8201 - recall: 0.0172 - auc: 0.8039 - prc: 0.3605 - val_loss: 0.8250 - val_tp: 6186.0000 - val_fp: 13559.0000 - val_tn: 137108.0000 - val_fn: 9725.0000 - val_accuracy: 0.9057 - val_precision: 0.7893 - val_recall: 0.0167 - val_auc: 0.7943 - val_prc: 0.3098
Epoch 10/10


2023-10-29 01:34:50.043449: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 230713 of 3000000
2023-10-29 01:35:00.043457: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 467466 of 3000000
2023-10-29 01:35:10.043434: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 703861 of 3000000
2023-10-29 01:35:20.043525: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 885892 of 3000000
2023-10-29 01:35:30.044232: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 911346 of 3000000
2023-10-29 01:35:40.047309: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 927793 of 3000000
2023-10-29 01:35:50.090017: I tensorflow/core/kernels/data/shuffle_dat

488/488 [==============================] - 231s 281ms/step - loss: 0.5655 - tp: 47145.0000 - fp: 81043.0000 - tn: 803189.0000 - fn: 66505.0000 - accuracy: 0.8878 - precision: 0.8360 - recall: 0.0182 - auc: 0.8072 - prc: 0.3656 - val_loss: 0.8490 - val_tp: 6516.0000 - val_fp: 14825.0000 - val_tn: 135842.0000 - val_fn: 9395.0000 - val_accuracy: 0.9058 - val_precision: 0.7896 - val_recall: 0.0191 - val_auc: 0.7922 - val_prc: 0.3093


In [33]:
tf.keras.models.save_model(model, filepath=saved_model_dir)

INFO:tensorflow:Assets written to: /Users/swapnil/work/stockdata/models/sequence/SEQUENCE_CONVOLUTIONAL/singlehead/saved_models/20231029-005935/assets


INFO:tensorflow:Assets written to: /Users/swapnil/work/stockdata/models/sequence/SEQUENCE_CONVOLUTIONAL/singlehead/saved_models/20231029-005935/assets


In [34]:
print(saved_model_dir)

/Users/swapnil/work/stockdata/models/sequence/SEQUENCE_CONVOLUTIONAL/singlehead/saved_models/20231029-005935


In [35]:
#tf.saved_model.save(model, saved_model_dir + "_1")

In [36]:
%load_ext tensorboard

In [37]:
#%tensorboard --logdir $tensorboard_dir
%tensorboard --logdir /Users/swapnil/work/stockdata/models/SINGLE_HEAD_CANDLE_STICKS_ONLY/singlehead/tensorboard/20221109-134059

Reusing TensorBoard on port 6007 (pid 39801), started 2 days, 0:04:36 ago. (Use '!kill 39801' to kill it.)